### Notebook for estimating the impact of modeling the leaf as homogeneous vs heterogeneous over a range of parameters

The data stored in data/saved_data/sensitivies.txt reflects the heatmap of sensitivity $\eta(\tau,\gamma)$ as displayed in figure 3C.

In [ ]:
import numpy as np 
from tqdm import tqdm # implementation of progress bars in jupyter notebooks
import sys 
sys.path.append('../../modules/')
import leaf_model as lm

In [ ]:
def sensitivity_search(delta_min=0.2, kappa_min=0.2, lambda_min=0.2, chi_=0.10, N=400, n=5):
    ''' 
    Function to perform a parameter search for the sensitivity \eta over parameters \tau, \gamma and structural parameters encoding the contrast between spongy and palisade mesophyll.
    delta_min: minimum value for spongy/palisade mesophyll contrast in effective IAS diffusivity rho_delta (D_PM/D_SM) [float in (0,1]]
    kappa_min: minimum value for spongy/palisade mesophyll contrast in absorption rate rho_kappa (K_SM/K_PM) [float in (0,1]]
    lambda_min: minimum value for fractional volume occupied by palisade mesophyll rho_lambda (L_PM/L) [float in (0,1)]
    chi_: relative CO2 compensation point C*/Ca [float]
    N: number of points in taus and gammas (N x N grid) [int]
    n: number of points in rho_deltas, rho_kappas, rho_lambdas (n x n x n grid) [int]
    Returns:
        taus: array of tau values (x-axis)
        gammas: array of gamma values (y-axis)
        sensitivities: 2D array of sensitivities \eta with shape (N, N) where sensitivities[i,j] corresponds to sensitivity for taus[i] and gammas[j]
    '''
    # set up mesh of parameters (tau, gamma), rho_deltas=D_PM/D_SM, rho_kappas=K_SM/K_PM, rho_lambdas=L_PM/L
    taus   = np.exp(np.linspace(np.log(0.01), np.log(100), N))
    gammas = np.exp(np.linspace(np.log(0.01), np.log(100), N))
    rho_deltas  = np.linspace(delta_min, 1, n) # from maximal to no contrast
    rho_kappas  = np.linspace(kappa_min, 1, n) # from maximal to no contrast
    rho_lambdas = np.linspace(lambda_min, 1-lambda_min, n) # symmetric around 0.5
    # initialize leaf model object 
    leaf = lm.Leaf(1, 1, chi_, rho=(1,1,0.5))
    sensitivities  = np.zeros((N,N)) # container for sensitivities \eta
    # loop over taus and gammas
    for i in tqdm(range(N)):
        for j in range(N):
            # update leaf model parameters
            leaf.tau   = taus[i]
            leaf.gamma = gammas[j]
            leaf.rho = (1,1,0.5) # homogeneous mesophyll parameters (no spongy/palisade mesophyll contrast)
            # calculate homogeneous mesophyll solution
            domain, mf_solution = leaf.calculate_steady_state_solution()
            #
            samples = n**3
            drawdown = np.zeros(samples) # containers for relative drawdown (Ca - Ci)/Ca = 1 - \chi_i values between homogeneous and heterogeneous mesophyll solutions
            # relative differences in relative drawdowns will reflect relative differences in assimilation rates because gs (and gamma) is constant between homogeneous and heterogeneous model
            index = 0
            # loop over rho_deltas, rho_kappas, rho_lambdas
            for k in range(n):
                for l in range(n):
                    for m in range(n):
                        rho = (rho_deltas[k], rho_kappas[l], rho_lambdas[m])
                        leaf.rho = rho
                        domain, solution = leaf.calculate_steady_state_solution() # calculate steady state solution for heterogeneous mesophyll model
                        drawdown[index] = (mf_solution[0] - solution[0]) / (1 - mf_solution[0]) # calculate relative difference in relative drawdown
                        index += 1
            sensitivities[i,j] = np.sqrt(np.sum(drawdown**2)/samples)
    return taus, gammas, sensitivities.T #


taus, gammas, sensitivities = sensitivity_search()
np.savetxt('../saved_data/sensitivities.txt', sensitivities, delimiter=';', header='delta_min=0.2, kappa_min=0.2, lambda_min=0.2, chi_=0.10, N=400, n=5')

